In [138]:
import pandas as pd
import numpy as np
import warnings

In [12]:
logs = pd.read_csv("./datasets/train_logs.csv")
logs

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,001519c8,1,4526,4557,31,Nonproduction,Leftclick,Leftclick,NoChange,0,0
1,001519c8,2,4558,4962,404,Nonproduction,Leftclick,Leftclick,NoChange,0,0
2,001519c8,3,106571,106571,0,Nonproduction,Shift,Shift,NoChange,0,0
3,001519c8,4,106686,106777,91,Input,q,q,q,1,1
4,001519c8,5,107196,107323,127,Input,q,q,q,2,1
...,...,...,...,...,...,...,...,...,...,...,...
8405893,fff05981,3615,2063944,2064440,496,Nonproduction,Leftclick,Leftclick,NoChange,1031,240
8405894,fff05981,3616,2064497,2064497,0,Nonproduction,Shift,Shift,NoChange,1031,240
8405895,fff05981,3617,2064657,2064765,108,Replace,q,q,q => q,1031,240
8405896,fff05981,3618,2069186,2069259,73,Nonproduction,Leftclick,Leftclick,NoChange,1028,240


In [148]:
from sklearn.feature_extraction.text import TfidfVectorizer

def process_changes(row):
    for i in range(len(row)):
        if "=>" in row[i]:
            row[i] = row[i].split('=>')[1]
        
        if row[i] == ' ':
            row[i] = 'Space'
    return row

def changes_extraction(ds):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)
    tfidf_vectorizer.fit(ds['text_change'])
    feature_names = tfidf_vectorizer.get_feature_names_out()
    
    word_counts_df = pd.DataFrame(columns=['id'] + feature_names)
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i, row in ds.iterrows():
            row_counts = {'id': row['id']}
            row_counts.update({word: row['text_change'].count(word) for word in feature_names})
            word_counts_df = word_counts_df.append(row_counts, ignore_index=True)
        
    return word_counts_df

In [149]:
changes_ds = logs[(logs['text_change'] != 'NoChange')].groupby('id')['text_change'].agg(list).agg(process_changes).apply(lambda x: " ".join(x)).reset_index()

In [150]:
changes_extraction(changes_ds)